In [1]:
from sympy import *


In [2]:
#calculated solution
z = symbols("z",real=True)
w,r,O,B = 1,1,1,1
A = 1/(-z**2*O**2 + I*r*z*O + w**2)
Z = A*E**(I*O*z)
solution = factor( re(A)*re(E**(I*O*z)) )

In [3]:
NeuralNetwork = 0
train_t = 0

project applied machine learning,VO Applied machine learning (2023W)

In [3]:
%%html
<style>

.t{
    display:flex;margin:5%;list-style-type:none;justify-content: space-evenly;
}
.pagebreak {
    clear: both;
    page-break-after: always;
}

</style>

<div class="pagebreak"></div>

<div class="t"> <li>12031612  </li><li> VO Applied machine learning (2023W)</li></div>


<hr>

<h2 style="text-align: center;">Project : Physics informed neural networks PINNs </h2>

<br>
<h3>Outline:</h3>
<div style="    
    list-style-type: none; 
    margin: 15px;">
    <li ><a href='#graphs'>a)What is a PINN</a></li>
    <li><a href='#osc'>b)example: harmonic oscilator</a></li>
    <div style="margin-left: 15px;" >
    <li><a href='#training'>i)training</a></li>
    <li> <a href='#CPINN'>ii)CPINN - Competitive PINNS </a></li>
    </div>
    <li> <a href='#ref'>references </a></li>

</div>
<br>

<h3>a)What is a PINN?</h3>
<a id='graphs'></a>


We are trying to approximate a particular solution of second order inhomegenous ODE 

$$
m * \ddot y + c * \dot y + k * y = g(t),
$$
with a neural network of the form

$$
{  L_{i} = \large \sigma_{i} \left({ \begin{bmatrix}
&  &  \\
& {\large W_{i}} & \\
&  &  \\
\end{bmatrix} * 
\begin{bmatrix}
  \\
{\large x_{i}}   \\
  \\
\end{bmatrix} +
\begin{bmatrix}
  \\
{\large b_{i}}   \\
 \\
\end{bmatrix} 
 }\right) }
$$
<br>
$$
N(t) =  (L_{1}  \circ L_{2}  \circ L_{3} \circ L_{o}) (t) 
$$
<!--  -->





where $\sigma_{i}$ is an yet undetermed activation function. and $L_o \rightarrow  \mathbb{R}$ is the output neuron of the form.
<br>
<br>
$$  L_{o} = {\large \sigma} \left( 
  \begin{bmatrix}
 & {\large w} & \\
\end{bmatrix} * 
\begin{bmatrix}
  \\
{\large x}   \\
  \\
\end{bmatrix} +
 \lambda \right)$$

 

Therefore we define a loss function
<br>
$$
\mathcal{L}(\mathbf{W}) = \frac{1}{M}\sum_{i=1}^M \underline{ (m * \ddot {N}(t_i) + c * \dot {N}(t_i) + k * {N}(t_i) - g(t) ) ^  2 }+ {({N}(0) - y_0) ^ 2 + (\dot {N}(0) - \dot y_0) ^ 2}
$$
Where sceond and third terms are the initial value losses. To minimize the loss function we choose Stochastic Gradient descent algorithm.
$$
--
$$

<h3 >b)example: harmonic oscilator</h3>
<a id='osc'> </a>
<h4>i)training</h4>
<a id='tr'> </a>




Consider the harmonic oscilator

$$
\ddot y + r * \dot y + \omega ^2 * y = B * cos(\Omega * t),
$$
for simplicity we set all parameters to 1,

$$
\ddot y + \dot y +  y = cos(t),
$$
and initialize the  $L_i$ ,
$$
{  L_{i} =  tanh \left({ \begin{bmatrix}
&  &  \\
& 32 \times 32 &\\
&  &  \\
\end{bmatrix} * 
\begin{bmatrix}
  \\
{\large x_{i}}   \\
  \\
\end{bmatrix} +
\begin{bmatrix}
  \\
32   \\
 \\
\end{bmatrix} 
 }\right) }
 $$
 <br>
with values in [0,1). The calculated solution of the equation is 
$$
y_p(t) = sin(t).
$$


In Python we can implement the traing loop for example with tensorflow.


In [5]:
def train(epochs = 300):
    trainable_vars = NeuralNetwork.trainable_variables()
    optimizer = tf.optimizers.SGD(learning_rate=0.01)
    for _ in range(epochs):
        with tf.GradientTape(persistent=True) as tape: 
            loss = loss(train_t,NeuralNetwork,g)
        grad = tape.gradient(loss, trainable_vars)
        optimizer.apply_gradients(grad, trainable_vars)

That is we calculate the gradient with respect to each variable and shift the variable in the direction of the gradient accordingly,

plotting the calculated solution and the predicted solution in <i style="color:blue">blue</i> for  values in  [0,2].
(the dots are training points)

<img src="png/output.png">

Calculating the error with respect to the $L_2 $ norm,with trapazoid integral yields

$$ || NeuralNetwork(t) - sin(t) ||_{L_2}  =  0.20278955 .$$


If we plot the loss, we can see its convergening to $ \approx0.28.$

<img src="png/loss.png">

$$
--
$$

<h4> ii)CPINN - Competitive PINNS </h4>
<a id='CPINN'> </a>

We are trying to tackle this further by adding a discriminator neural network $ T(t) $ ,
$$ \mathcal{L}(N,T,t) = \frac{1}{M}\sum_{i=1}^M  T(t) * ( \ddot N(t_i) + \dot N(t_i) + N(t_i) )^2$$

 with the according optimization problem,
$$ max_T \phantom,   min_N \phantom,   \mathcal{L}(N,T,t). $$

We try stochastic gradient descent for T and for N, that is  $  SGD_{learnable variables \phantom,  L }( - \mathcal{L}(N,T,t)) $ and $ SGD_{learnable variables\phantom, NN }( \mathcal{L}(N,T,t))$ for each iteration.

Plotting the loss we see it seems to converge to the 'Nash equilibrium'.    $ (T,NN) \rightarrow (0,y) $  which would be a solution for the equation. 
This is not expected because trying to apply SGD seperatly just works for specific zero sum optimization problems.

<img src="png/losscpin.png">

the resulting loss plot and $L_2$ norm is now

$$ || NeuralNetwork(t) - sin(t) ||_{L_2}  = 0.18570352  $$

One has to consider Optimizers for zero sum problems  for example Competitive Gradient Descent or Sympletic Gradient Adjustment.

<h4 >references</h4>
<a id='ref'> </a>